In [1]:
import numpy as np
import pandas as pd
from math import radians, cos, sin, asin, sqrt
import random
from ortools.sat.python import cp_model
import time

In [2]:
#---------------------------------- EXTRACT THE DATA ----------------------------------
start_time = time.time()
# users-melbcbd-generated.csv contains:
# •  Latitude-Longitude
# of the users in the Melbourne CBD area.
users_path = '..\\eua-dataset\\users\\'
#U = pd.read_csv(users_path + 'users-test.csv') # test with 8 users
#U = pd.read_csv(users_path + 'users-test-more-users.csv') # test with 14 users
U = pd.read_csv(users_path + 'users-melbcbd-generated.csv') # test with 816 users

# site-optus-melbCBD.csv contains:
# •  SiteID-Latitude-Longitude-Name-State-LicensingAreaID-PostCode-SitePrecision-Elevation-HCISL2
# of all Optus BS in Melbourne CBD area (edge-servers)
servers_path = '..\\eua-dataset\\edge-servers\\'
#S = pd.read_csv(servers_path + 'serverstest.csv') # test with 3 servers
#S = pd.read_csv(servers_path + 'serverstest-more-servers.csv') # test with 6 servers
S = pd.read_csv(servers_path + 'site-optus-melbCBD.csv') # test with 125 servers

In [3]:

#-----------------------------------VARIABLES-----------------------------------
# All the requirements and capacities are randomly generated from a set of possibilites

dij = []  # Distance between server i and user j
D = np.zeros((4), dtype=int) # D = [CPU, RAM, storage, bandwidth]

# User u can have 3 different types of memory requirements = w
w1 = [1, 2, 1, 2]
w2 = [2, 3,	3, 4]
w3 = [5, 7,	6, 6]

W = []
for j in range(len(U)):
  w = random.choice([w1, w2, w3])
  W.append(w)

# Each server has memory capacity c
mu, sigma = 35, 10 # mean and standard deviation

C = []
for j in range(len(S)):
  norm_dist = np.random.normal(mu, sigma**2, len(D))
  c = norm_dist.astype(int)
  for i in range(len(c)):
    if c[i] < 0:
      c[i] = 1 # Value suggested by MCF paper in case of negative values
  C.append(c)


In [4]:
#---------------------------------- COVERAGE OF EACH SERVER ----------------------------------

def haversine(lon1, lat1, lon2, lat2):
  
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

radius = np.round(np.random.uniform(0.45,0.75,len(S)),3) # in Km
#radius = np.round(np.random.uniform(0.1,0.15,len(S)),3) # in km (values comparable with MCF)
#radius = np.full(len(S), 0.03)

for i in range(len(S)):
  server_latitude = S.iloc[i]['LATITUDE']
  server_longitude = S.iloc[i]['LONGITUDE']
  dist_temp = []
  for j in range(len(U)):
    user_latitude = U.iloc[j]['Latitude']
    user_longitude = U.iloc[j]['Longitude']

    dist_geo = haversine(server_longitude, server_latitude, user_longitude, user_latitude) # Distance between User and Server
    dist_temp.append(np.round(dist_geo,3))

  dij.append(dist_temp)

In [5]:
#---------------------------------- CP MODEL ----------------------------------
model = cp_model.CpModel()

#---------------------------------- SOLVER VARIABLES ----------------------------------
# x_i,j = True if user u_j is allocated to edge server s_i
# x_i,j = False otherwise
x = {}
for j in range(len(U)):
    for i in range(len(S)):
        x[i, j] = model.NewBoolVar(f'c[{i}][{j}]')

# y_i = True if edge server s_i is used to serve users
# y_i = False otherwise
y = {}
for i in range(len(S)):
    y[i] = model.NewBoolVar(f'c[{i}]')

In [6]:
#---------------------------------- CONSTRAINTS ----------------------------------

# Capacity constraint
for i in range(len(S)):
    for k in range(len(D)):
        model.Add(
            sum([
                x[i, j] * W[j][k] for j in range(len(U))
            ]) <= C[i][k]*y[i]
        )

# Proximity constraint
for j in range(len(U)):
    for i in range(len(S)): 
        if dij[i][j]>= radius[i]:
            model.Add(
                x[i,j]==0
            )

# Contraint family (each user can be allocated just once)
for j in range(len(U)):
    model.Add(
        sum([
            x[i, j] for i in range(len(S))
        ]) <= 1
    )
        
print(model.ModelStats())

satisfaction model '':
#Variables: 102125
  - 102125 Booleans in [0,1]
#kLinear1: 61036
#kLinearN: 1316 (#terms: 510494)


In [7]:
#---------------------------------- CP SOLVER ----------------------------------
solver = cp_model.CpSolver()

#---------------------------------- OBJECTIVE FUNCTIONS ---------------------------------

# Maximize the number of allocated users
objective_max = []
for j in range(len(U)):
    for i in range(len(S)):
        objective_max.append(x[i, j])
model.Maximize(sum(objective_max))

solver.Solve(model)
max_users = solver.ObjectiveValue()

# Hint (speed up solving)
for j in range(len(U)):
    for i in range(len(S)):
        model.AddHint(x[i,j], solver.Value(x[i,j]))

for i in range(len(S)):
    model.AddHint(y[i], solver.Value(y[i]))
        
# Constraint previous objective
model.Add(
    sum([
        x[i, j] for i in range(len(S)) for j in range(len(U))
    ]) == round(solver.ObjectiveValue())
)

# Minimize the number of servers used
objective_min = []
for i in range(len(S)):
    objective_min.append(y[i])
model.Minimize(sum(objective_min))

print(model.ModelStats())

optimization model '':
#Variables: 102125 (#bools:125 in objective)
  - 102125 Booleans in [0,1]
#kLinear1: 61036
#kLinearN: 1317 (#terms: 612494)


In [8]:
#---------------------------------- CALL THE SOLVER ----------------------------------
status = solver.Solve(model)

#---------------------------------- DISPLAY THE SOLUTION ----------------------------------

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Solution:')
    print(f'Objective value: {max_users} users have been allocated to {solver.ObjectiveValue()} servers\n')
    for j in range(len(U)):
        for i in range(len(S)):
            if int(solver.Value(x[i,j])) == 1:
                print(f'User {j+1} has been allocated on node {i+1}')
    
else:
    print('The problem does not have an optimal solution.\n')

print("\n--- Run time: %s seconds ---" % round((time.time() - start_time),2))

Solution:
Objective value: 469.0 users have been allocated to 59.0 servers

User 1 has been allocated on node 4
User 3 has been allocated on node 27
User 5 has been allocated on node 89
User 6 has been allocated on node 29
User 8 has been allocated on node 73
User 9 has been allocated on node 123
User 11 has been allocated on node 113
User 12 has been allocated on node 110
User 13 has been allocated on node 123
User 14 has been allocated on node 21
User 15 has been allocated on node 48
User 16 has been allocated on node 48
User 19 has been allocated on node 123
User 21 has been allocated on node 119
User 23 has been allocated on node 119
User 25 has been allocated on node 4
User 27 has been allocated on node 29
User 28 has been allocated on node 21
User 29 has been allocated on node 21
User 30 has been allocated on node 123
User 34 has been allocated on node 10
User 38 has been allocated on node 76
User 39 has been allocated on node 106
User 41 has been allocated on node 27
User 42 has

In [9]:
'''
#---------------------------------- VARIABLES ----------------------------------

dij = []  #Distance between server i and user j

D = np.zeros((4), dtype=int) # D = [CPU, disk I/O,.....]

#User u has memory requirements = w (example for 8 users)
w1 = [1, 2, 1, 2]
w2 = [1, 2,	1, 2]
w3 = [5, 7,	6, 6]
w4 = [2, 3,	3, 4]
w5 = [5, 7,	6, 6]
w6 = [2, 3,	3, 4]
w7 = [1, 2, 1, 2]
w8 = [2, 3,	3, 4]
w9 = [1, 2, 1, 2]
w10 = [1, 2, 1, 2]
w11 = [5, 7, 6, 6]
w12 = [2, 3, 3, 4]
w13 = [5, 7, 6, 6]
w14 = [2, 3, 3, 4]
W = [w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, w14]

# Each server has memory capacity c (example for 3 servers)
c1 = [12, 15, 15, 16]
c2 = [10, 17, 15, 16]
c3 = [10, 15, 11, 15]
c4 = [10, 16, 15, 17]
c5 = [12, 12, 12, 13]
c6 = [9, 13, 12, 14]
C = [c1, c2, c3, c4, c5, c6]
'''

'\n#---------------------------------- VARIABLES ----------------------------------\n\ndij = []  #Distance between server i and user j\n\nD = np.zeros((4), dtype=int) # D = [CPU, disk I/O,.....]\n\n#User u has memory requirements = w (example for 8 users)\nw1 = [1, 2, 1, 2]\nw2 = [1, 2,\t1, 2]\nw3 = [5, 7,\t6, 6]\nw4 = [2, 3,\t3, 4]\nw5 = [5, 7,\t6, 6]\nw6 = [2, 3,\t3, 4]\nw7 = [1, 2, 1, 2]\nw8 = [2, 3,\t3, 4]\nw9 = [1, 2, 1, 2]\nw10 = [1, 2, 1, 2]\nw11 = [5, 7, 6, 6]\nw12 = [2, 3, 3, 4]\nw13 = [5, 7, 6, 6]\nw14 = [2, 3, 3, 4]\nW = [w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, w14]\n\n# Each server has memory capacity c (example for 3 servers)\nc1 = [12, 15, 15, 16]\nc2 = [10, 17, 15, 16]\nc3 = [10, 15, 11, 15]\nc4 = [10, 16, 15, 17]\nc5 = [12, 12, 12, 13]\nc6 = [9, 13, 12, 14]\nC = [c1, c2, c3, c4, c5, c6]\n'

In [10]:
'''
#---------------------------------- VARIABLES ----------------------------------

dij = []  #Distance between server i and user j

D = np.zeros((4), dtype=int) # D = [CPU, disk I/O,.....]

# Users u has memory requirements = w_j
W = []
for j in range(len(U)):
  w = np.random.randint(1,7,4)
  W.append(w)

# Each server has memory capacity C
C = []
for j in range(len(S)):
  c = np.random.randint(9,17,4)
  C.append(c)
'''

'\n#---------------------------------- VARIABLES ----------------------------------\n\ndij = []  #Distance between server i and user j\n\nD = np.zeros((4), dtype=int) # D = [CPU, disk I/O,.....]\n\n# Users u has memory requirements = w_j\nW = []\nfor j in range(len(U)):\n  w = np.random.randint(1,7,4)\n  W.append(w)\n\n# Each server has memory capacity C\nC = []\nfor j in range(len(S)):\n  c = np.random.randint(9,17,4)\n  C.append(c)\n'